In [1]:
import copy
import datetime
import random
import time

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from transformers import BertForSequenceClassification, AdamW
from transformers import BertTokenizer
from transformers import get_linear_schedule_with_warmup

#from reddit_preprocessing import MAX_SEQ_LENGTH, AHOLE_CLASSES, load_dataset

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [4]:
import string
import re
import pandas as pd

URL_REGEX = '(http|https)://[-a-zA-Z0-9+&@#/%?=~_|!:,.;]*[-a-zA-Z0-9+&@#/%=~_|]'
NON_ALPHA_NUMERIC_REGEX = '[^a-zA-Z0-9- ]'
TAG_REGEX = '@[^\\s]*'

NUM_CLASSES = 2
AHOLE_CLASSES = ['yta', 'nta']
MAX_SEQ_LENGTH = 64


def load_dataset(filepath: str, classes: list, tokenizer, rm_punct: bool = False):
    texts, labels = [], []
    df = pd.read_csv(filepath)
    df = df.head(50)
    
    df_yta = df[df['is_yta'] == 'yta']
    df_nta = df[df['is_yta'] == 'nta']
    #n = df_yta.shape[0]
    #df_nta = df_nta.iloc[:n]
    
    #df = pd.concat([df_yta, df_nta], axis=0)
    print(df)

    
    df['body'] = df['body'].astype("str")
    df['is_yta'] = df['is_yta'] == "yta"
    for index, row in df.iterrows():
        label = row['is_yta']
        text = row['body']
        tokens = process_text(text, tokenizer, rm_punct)
        if len(tokens) > 3:
            texts.append(tokens)
            labels.append([label, not label])

    return texts, labels
def process_text(text, tokenizer, rm_punct: bool = False):
    if type(text) is not str:
        raise TypeError('Text is not of type string')

    # remove special string from the text: URLs and emojis (by encoding and decoding to/from ascii)
    text = remove_urls(text)
    text = remove_tags(text)

    # remove unknown characters
    text = text.encode('ascii', 'ignore').decode('ascii')
    # remove all punctuation
    if rm_punct:
        text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = tokenizer.tokenize(text)
    # remove all labels from texts
    tokens = delete_label_word(tokens, 'YTA')
    tokens = delete_label_word(tokens, 'NTA')

    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    return tokens


def remove_non_alphanumeric(text: str) -> str:
    return re.sub(NON_ALPHA_NUMERIC_REGEX, ' ', text)


def remove_urls(text: str) -> str:
    return re.sub(URL_REGEX, ' ', text)


def remove_tags(text: str) -> str:
    return re.sub(TAG_REGEX, ' ', text)


def delete_label_word(words: list, label: str) -> list:
    while label in words:
        words.remove(label)
    return words

In [5]:
texts, labels = load_dataset(filepath='/Users/teodorareu/PycharmProjects/COMP34812_k01092tr/data/posts.csv',
                             classes=AHOLE_CLASSES,
                             tokenizer=tokenizer,
                             rm_punct=True)
yta_nr = labels.count([True,False])
nta_nr = len(labels) - yta_nr
print(yta_nr)
print(nta_nr)

        id                                              title  \
0   eiawts  AITA for Not Letting My Friend Invite Her Auti...   
1   eiaw8a  WIBTA if I don't pick my cousin up tonight fro...   
2   eiav15                      AITA for disowning my family?   
3   eiau2i  AITA for “pulling rank” on someone who was giv...   
4   eiatrs  AITA for sleeping on the same bed with my friend?   
5   eiasoj  AITA for freaking out on my mom for not signin...   
6   eias5l     AITA for ghosting a guy that has a crush on me   
7   eiaoo9  AITA for telling my family that my sister woul...   
8   eiagfd   AITA for cutting ties with an unreliable friend?   
9   eiafwb  AITA for not paying my cousin back right away ...   
10  eiae06  AITA for not having my mother not in my child'...   
11  eiacb2  AITA for cutting off my aunt for spreading mal...   
12  eiaady  AITA for having a secret credit card that I hi...   
13  eia7fa  AITA for not wanting to have restrictions from...   
14  eia774  AITA for maki

In [6]:
input_ids, attention_masks = [], []
for text in texts:
    encoded_dict = tokenizer.encode_plus(text,
                                         add_special_tokens=False,
                                         truncation=True,
                                         max_length=MAX_SEQ_LENGTH,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt')
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

/Users/teodorareu/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [7]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [8]:
labels = labels.bool().int().float()

In [9]:
input_ids.shape

torch.Size([50, 64])

In [10]:
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

batch_size = 2

train_dataloader = DataLoader(train_dataset,  # The training samples.
                              sampler=RandomSampler(train_dataset),  # Select batches randomly
                              batch_size=batch_size)

validation_dataloader = DataLoader(val_dataset,  # The validation samples.
                                   sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
                                   batch_size=batch_size)

   37 training samples
   13 validation samples


In [11]:
validation_dataloader.dataset[3]

(tensor([  101,  2061,  2182,  2015,  1037,  2210, 10457,  7062, 10047,  1037,
          5014,  9905,  9890,  2094,  4845,  2026,  2972,  2166,  2287,  2385,
          1045,  2147,  2012, 14556, 10882,  2140, 18887,  2031,  5552,  2039,
          1037,  3232,  4595,  6363,  2013,  2147,  1045,  2525,  2031,  4149,
          2870,  1037,  6942,  1998,  1037, 26381,  2029,  2026,  3008,  9038,
          4352,  2033,  2000,  4965,  2055,  1018,  2706,  3283,  1045,  2787,
          1045,  2359,  1037, 10355]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([0., 1.]))

In [12]:

from models import WeightedBCEBert

model = WeightedBCEBert()

optimizer = AdamW(model.parameters(),
                  lr=2e-5,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps=1e-8  # args.adam_epsilon  - default is 1e-8.
                  )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/teodorareu/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and

In [13]:
epochs = 4

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=100,
                                            num_training_steps=total_steps)


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten().astype(np.float32)
    labels_flat = labels[:,1].flatten().astype(np.float32)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [14]:
p = torch.arange(64, dtype=torch.int64).unsqueeze(0)
l = torch.ones(64).unsqueeze(0)
weights = torch.Tensor([1,1])
labels = torch.ones(1,2)
m = model(p, l, None, labels, weights)

In [15]:
print(m)

SequenceClassifierOutput(loss=tensor(0.8414, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), logits=tensor([[-0.0447, -0.4886]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


In [19]:
# TRAINING
# Tell pytorch to run this model on the GPU.
#model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()

best_val_acc, best_val_loss = 0, torch.finfo(torch.float32).max

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    total_train_accuracy = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 400 batches.
        if step % 10 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        b_weights = torch.Tensor([yta_nr, nta_nr])
        outputs = model(b_input_ids,b_input_mask, None, b_labels,b_weights)

        loss = outputs.loss
        logits = outputs.logits
        total_train_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of train sentences, and accumulate it over all batches.
        total_train_accuracy += flat_accuracy(logits, label_ids)

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Report the final accuracy for this train run.
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Training accuracy: {0:.4f}".format(avg_train_accuracy))
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode -- the dropout layers behave differently during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            model.zero_grad()
            b_weights = torch.Tensor([yta_nr, nta_nr])
            outputs = model(b_input_ids,b_input_mask, None, b_labels,b_weights)

        loss = outputs.loss
        logits = outputs.logits
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    # save the model with the best accuracy and minimal loss (so we do not save an overfitted model)
    if avg_val_accuracy > best_val_acc and avg_val_loss < best_val_loss:
        best_val_acc = avg_val_accuracy
        best_val_loss = avg_val_loss
        best_model = copy.deepcopy(model)
        best_epoch = epoch_i

    print("  Validation Acc.: {0:.4f}".format(avg_val_accuracy))
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Training Acc.': avg_train_accuracy * 100,
            'Valid. Loss': avg_val_loss,
            'Valid. Acc.': avg_val_accuracy * 100,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    10  of     19.    Elapsed: 0:00:16.

  Training accuracy: 0.3684
  Average training loss: 18.1945
  Training epcoh took: 0:00:29

Running Validation...
  Validation Acc.: 0.6429
  Validation Loss: 10.1823
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...
  Batch    10  of     19.    Elapsed: 0:00:16.

  Training accuracy: 0.8947
  Average training loss: 11.4014
  Training epcoh took: 0:00:30

Running Validation...
  Validation Acc.: 0.6429
  Validation Loss: 7.1095
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...
  Batch    10  of     19.    Elapsed: 0:00:16.

  Training accuracy: 0.9211
  Average training loss: 4.9715
  Training epcoh took: 0:00:30

Running Validation...
  Validation Acc.: 0.6429
  Validation Loss: 2.6536
  Validation took: 0:00:02

======== Epoch 4 / 4 ========
Training...
  Batch    10  of     19.    Elapsed: 0:00:16.

  Training accuracy: 0.9211
  Average training loss: 1

In [ ]:
best_model.save_pretrained('/models')
print('Best fit model saved at epoch %d' % (best_epoch + 1))

In [ ]:
torch.save(model.state_dict(), "./best_model_posts.pt")


In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 4)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training Loss")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation Loss")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks(list(range(1, epochs + 1)))

plt.show()